In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import json

import time
import random

In [2]:
pd.options.display.max_columns= None

In [7]:
dff = pd.read_csv('nfl_data.csv')
dff

,season,team,Week,Day,Date,Unnamed: 3,Unnamed: 4,OT,Unnamed: 6,Opp,Tm,Opp.1,Cmp,Att,Yds,TD,Int,Sk,Yds.1,Y/A,NY/A,Cmp%,Rate,Att.1,Yds.2,Y/A.1,TD.1,FGM,FGA,XPM,XPA,Pnt,Yds.3,3DConv,3DAtt,4DConv,4DAtt,ToP,Week.1,Day.1,Date.1,Unnamed: 3.1,Unnamed: 4.1,OT.1,Unnamed: 6.1,Opp.2,Tm.1,Opp.1.1,Cmp.1,Att.2,Yds.4,TD.2,Int.1,Sk.1,Yds.1.1,Y/A.2,NY/A.1,Cmp%.1,Rate.1,Att.1.1,Yds.2.1,Y/A.1.1,TD.1.1,FGM.1,FGA.1,XPM.1,XPA.1,Pnt.1,Yds.3.1,3DConv.1,3DAtt.1,4DConv.1,4DAtt.1,ToP.1
0,2000,CRD,1,Sun,September 3,boxscore,L,NaN,@,New York Giants,16,21,28,49,312,2,3,1,6,6.5,6.2,57.1,64.3,20,43,2.2,0,1,1,1,1,5,210,6,15,0,2,28:30,1,Sun,September 3,boxscore,L,NaN,@,New York Giants,16,21,17,25,172,0,1,0,0,6.9,6.9,68.0,70.7,41,223,5.4,3,0,0,3,3,6,155,7,14,0,0,31:30
1,2000,CRD,2,Sun,September 10,boxscore,W,NaN,NaN,Dallas Cowboys,32,31,18,24,224,2,0,2,19,10.1,8.6,75.0,131.2,33,98,3.0,1,4,4,2,2,2,94,7,12,0,0,31:20,2,Sun,September 10,boxscore,W,NaN,NaN,Dallas Cowboys,32,31,24,34,240,3,0,1,3,7.1,6.9,70.6,119.7,22,90,4.1,0,1,1,4,4,2,23,5,12,3,4,28:40
2,2000,CRD,4,Sun,September 24,boxscore,L,NaN,NaN,Green Bay Packers,3,29,21,43,181,0,4,1,8,4.4,4.1,48.8,21.6,13,28,2.2,0,1,1,0,0,6,288,6,14,0,1,23:32,4,Sun,September 24,boxscore,L,NaN,NaN,Green Bay Packers,3,29,18,32,279,1,0,1,9,9.0,8.5,56.3,95.7,36,176,4.9,1,5,5,2,2,2,164,5,12,0,0,36:28
3,2000,CRD,5,Sun,October 1,boxscore,L,NaN,@,San Francisco 49ers,20,27,24,41,239,1,0,0,0,5.8,5.8,58.5,83.3,24,126,5.3,1,2,3,2,2,4,176,3,13,2,3,27:50,5,Sun,October 1,boxscore,L,NaN,@,San Francisco 49ers,20,27,22,33,215,1,0,1,5,6.7,6.3,66.7,94.9,32,130,4.1,2,2,2,3,3,6,137,7,16,0,0,32:10
4,2000,CRD,6,Sun,October 8,boxscore,W,NaN,NaN,Cleveland Browns,29,21,17,30,169,2,0,1,2,5.7,5.5,56.7,95.0,31,146,4.7,1,3,3,2,2,3,144,2,10,1,2,31:03,6,Sun,October 8,boxscore,W,NaN,NaN,Cleveland Browns,29,21,16,22,136,0,0,1,2,6.3,5.9,72.7,88.4,31,104,3.4,3,0,1,3,3,7,206,5,14,0,1,28:57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12345,2023,WAS,13,Sun,December 3,boxscore,L,NaN,NaN,Miami Dolphins,15,45,12,23,107,0,1,3,20,5.5,4.1,52.2,46.8,28,138,4.9,2,0,1,1,1,5,214,6,14,1,1,27:44,13,Sun,December 3,boxscore,L,NaN,NaN,Miami Dolphins,15,45,19,25,283,2,0,0,0,11.3,11.3,76.0,139.2,34,123,3.6,3,1,1,6,6,3,127,7,13,2,2,32:16
12346,2023,WAS,15,Sun,December 17,boxscore,L,NaN,@,Los Angeles Rams,20,28,19,36,218,3,1,1,8,6.3,5.9,52.8,87.5,21,79,3.8,0,0,0,2,3,5,203,3,14,3,5,24:07,15,Sun,December 17,boxscore,L,NaN,@,Los Angeles Rams,20,28,25,33,249,2,0,3,9,7.8,6.9,75.8,116.9,37,196,5.3,1,2,3,2,2,3,134,9,16,0,0,35:53
12347,2023,WAS,16,Sun,December 24,boxscore,L,NaN,@,New York Jets,28,30,16,35,143,1,2,1,13,4.5,4.0,45.7,42.9,22,102,4.6,3,0,0,4,4,8,294,3,12,1,1,23:44,16,Sun,December 24,boxscore,L,NaN,@,New York Jets,28,30,27,49,217,1,1,0,0,4.4,4.4,55.1,64.8,36,164,4.6,2,3,3,3,3,6,292,4,16,1,2,36:16
12348,2023,WAS,17,Sun,December 31,boxscore,L,NaN,NaN,San Francisco 49ers,10,27,17,28,163,1,2,1,6,6.0,5.6,60.7,59.1,14,62,4.4,0,1,1,1,1,3,156,3,9,0,1,21:47,17,Sun,December 31,boxscore,L,NaN,NaN,San Francisco 49ers,10,27,22,28,224,2,0,1,6,8.2,7.7,78.6,123.8,39,184,4.7,1,2,2,3,3,2,76,4,9,0,1,38:13


In [5]:
#Columns to be deleted
#df['Unnamed: 3'] / columns [38:48]

df.drop(df.columns[38:48], axis=1, inplace=True)

df.drop(columns= ['Unnamed: 3'], axis = 1, inplace=True)

In [6]:
# Columns to be renamed
#df['Unnamed; 4', 'Unnamed: 6']

df.columns = df.columns.str.lower()

df.rename(columns={'unnamed: 4': 'result','unnamed: 6': 'host', 'tm':'pts','opp.1':'opp_pts',
                   'cmp':'pass_cmp', 'att':'pass_att','yds':'pass_yds','td':'pass_td','yds.1':'yds_lost_sks',
                   'y/a':'yds_pass_att', 'ny/a':'net_yds_pass_att', 'rate':'pass_rating', 'att.1':'rush_att',
                   'yds.2': 'rush_yds', 'y/a.1':'rush_yds_att','td.1':'rush_td','yds.3':'punt_yds'}, inplace = True)



# List of original column names
renamed_columns = ['pass_cmp', 'pass_att', 'pass_yds', 'pass_td', 'int', 'sk', 'yds_lost_sks', 'yds_pass_att', 
                    'net_yds_pass_att', 'cmp%', 'pass_rating', 'rush_att', 'rush_yds', 'rush_yds_att', 
                    'rush_td', 'fgm', 'fga', 'xpm', 'xpa', 'pnt', 'punt_yds', '3dconv', '3datt', 
                    '4dconv', '4datt', 'top']

# Add '_opp' to each name
new_column_names = [col + '_opp' for col in renamed_columns]

# Renaming columns in the DataFrame
df.rename(columns=dict(zip(df.columns[37:], new_column_names)), inplace = True)

In [7]:
df.columns

Index(['season', 'team', 'week', 'day', 'date', 'result', 'ot', 'host', 'opp',
       'pts', 'opp_pts', 'pass_cmp', 'pass_att', 'pass_yds', 'pass_td', 'int',
       'sk', 'yds_lost_sks', 'yds_pass_att', 'net_yds_pass_att', 'cmp%',
       'pass_rating', 'rush_att', 'rush_yds', 'rush_yds_att', 'rush_td', 'fgm',
       'fga', 'xpm', 'xpa', 'pnt', 'punt_yds', '3dconv', '3datt', '4dconv',
       '4datt', 'top', 'pass_cmp_opp', 'pass_att_opp', 'pass_yds_opp',
       'pass_td_opp', 'int_opp', 'sk_opp', 'yds_lost_sks_opp',
       'yds_pass_att_opp', 'net_yds_pass_att_opp', 'cmp%_opp',
       'pass_rating_opp', 'rush_att_opp', 'rush_yds_opp', 'rush_yds_att_opp',
       'rush_td_opp', 'fgm_opp', 'fga_opp', 'xpm_opp', 'xpa_opp', 'pnt_opp',
       'punt_yds_opp', '3dconv_opp', '3datt_opp', '4dconv_opp', '4datt_opp',
       'top_opp'],
      dtype='object')

In [8]:
df['host'].isnull().sum()

np.int64(5903)

In [11]:
df['host'].value_counts()

host
away    5903
home    5903
Name: count, dtype: int64

In [128]:
df['host'] = df['host'].fillna('home')
df['host'] = df['host'].replace('@','away')

#easier way to write same piece of code as above
df['host'] = df['host'].apply(lambda x: 'away' if x == '@' else 'home')

In [19]:
#replace nan values with yes or no values accoriding to ot results
df['ot'] = df['ot'].apply(lambda x: 'yes' if x == 'OT' else ' no')

In [20]:
df['ot'].value_counts()

ot
 no    11082
yes      724
Name: count, dtype: int64

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11806 entries, 0 to 11805
Data columns (total 63 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   season                11806 non-null  int64  
 1   team                  11806 non-null  object 
 2   week                  11806 non-null  int64  
 3   day                   11806 non-null  object 
 4   date                  11806 non-null  object 
 5   result                11806 non-null  object 
 6   ot                    11806 non-null  object 
 7   host                  11806 non-null  object 
 8   opp                   11806 non-null  object 
 9   pts                   11806 non-null  int64  
 10  opp_pts               11806 non-null  int64  
 11  pass_cmp              11806 non-null  int64  
 12  pass_att              11806 non-null  int64  
 13  pass_yds              11806 non-null  int64  
 14  pass_td               11806 non-null  int64  
 15  int                

In [5]:
def clean_data(csv_data):

    # read in data in form of csv data
    df = pd.read_csv(csv_data)
    
    #drop repeated columns information
    df.drop(df.columns[38:48], axis=1, inplace=True)
    
    #drop unnecessary column of 'boxscore'
    df.drop(columns= ['Unnamed: 3'], axis = 1, inplace=True)
    
    #lowercase all column names
    df.columns = df.columns.str.lower()
    
    #rename column names
    df.rename(columns={'unnamed: 4': 'result','unnamed: 6': 'host', 'tm':'pts','opp.1':'opp_pts',
                       'cmp':'pass_cmp', 'att':'pass_att','yds':'pass_yds','td':'pass_td','yds.1':'yds_lost_sks',
                       'y/a':'yds_pass_att', 'ny/a':'net_yds_pass_att', 'rate':'pass_rating', 'att.1':'rush_att',
                       'yds.2': 'rush_yds', 'y/a.1':'rush_yds_att','td.1':'rush_td','yds.3':'punt_yds'}, inplace = True)



    # List of original column names
    renamed_columns = ['pass_cmp', 'pass_att', 'pass_yds', 'pass_td', 'int', 'sk', 'yds_lost_sks', 'yds_pass_att', 
                        'net_yds_pass_att', 'cmp%', 'pass_rating', 'rush_att', 'rush_yds', 'rush_yds_att', 
                        'rush_td', 'fgm', 'fga', 'xpm', 'xpa', 'pnt', 'punt_yds', '3dconv', '3datt', 
                        '4dconv', '4datt', 'top']

    # Add '_opp' to each name
    new_column_names = [col + '_opp' for col in renamed_columns]

    # Renaming columns in the DataFrame
    df.rename(columns=dict(zip(df.columns[37:], new_column_names)), inplace = True)
    
    #replace nan values with home and @ with away to show which are home games or away
    df['host'] = df['host'].apply(lambda x: 'away' if x == '@' else 'home')
    
    #replace nan values with yes or no values accoriding to ot results
    df['ot'] = df['ot'].apply(lambda x: 'yes' if x == 'OT' else ' no')
    
    return df

In [6]:
df = clean_data('nfl_data.csv')
df

,season,team,week,day,date,result,ot,host,opp,pts,opp_pts,pass_cmp,pass_att,pass_yds,pass_td,int,sk,yds_lost_sks,yds_pass_att,net_yds_pass_att,cmp%,pass_rating,rush_att,rush_yds,rush_yds_att,rush_td,fgm,fga,xpm,xpa,pnt,punt_yds,3dconv,3datt,4dconv,4datt,top,pass_cmp_opp,pass_att_opp,pass_yds_opp,pass_td_opp,int_opp,sk_opp,yds_lost_sks_opp,yds_pass_att_opp,net_yds_pass_att_opp,cmp%_opp,pass_rating_opp,rush_att_opp,rush_yds_opp,rush_yds_att_opp,rush_td_opp,fgm_opp,fga_opp,xpm_opp,xpa_opp,pnt_opp,punt_yds_opp,3dconv_opp,3datt_opp,4dconv_opp,4datt_opp,top_opp
0,2000,CRD,1,Sun,September 3,L,no,away,New York Giants,16,21,28,49,312,2,3,1,6,6.5,6.2,57.1,64.3,20,43,2.2,0,1,1,1,1,5,210,6,15,0,2,28:30,17,25,172,0,1,0,0,6.9,6.9,68.0,70.7,41,223,5.4,3,0,0,3,3,6,155,7,14,0,0,31:30
1,2000,CRD,2,Sun,September 10,W,no,home,Dallas Cowboys,32,31,18,24,224,2,0,2,19,10.1,8.6,75.0,131.2,33,98,3.0,1,4,4,2,2,2,94,7,12,0,0,31:20,24,34,240,3,0,1,3,7.1,6.9,70.6,119.7,22,90,4.1,0,1,1,4,4,2,23,5,12,3,4,28:40
2,2000,CRD,4,Sun,September 24,L,no,home,Green Bay Packers,3,29,21,43,181,0,4,1,8,4.4,4.1,48.8,21.6,13,28,2.2,0,1,1,0,0,6,288,6,14,0,1,23:32,18,32,279,1,0,1,9,9.0,8.5,56.3,95.7,36,176,4.9,1,5,5,2,2,2,164,5,12,0,0,36:28
3,2000,CRD,5,Sun,October 1,L,no,away,San Francisco 49ers,20,27,24,41,239,1,0,0,0,5.8,5.8,58.5,83.3,24,126,5.3,1,2,3,2,2,4,176,3,13,2,3,27:50,22,33,215,1,0,1,5,6.7,6.3,66.7,94.9,32,130,4.1,2,2,2,3,3,6,137,7,16,0,0,32:10
4,2000,CRD,6,Sun,October 8,W,no,home,Cleveland Browns,29,21,17,30,169,2,0,1,2,5.7,5.5,56.7,95.0,31,146,4.7,1,3,3,2,2,3,144,2,10,1,2,31:03,16,22,136,0,0,1,2,6.3,5.9,72.7,88.4,31,104,3.4,3,0,1,3,3,7,206,5,14,0,1,28:57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12345,2023,WAS,13,Sun,December 3,L,no,home,Miami Dolphins,15,45,12,23,107,0,1,3,20,5.5,4.1,52.2,46.8,28,138,4.9,2,0,1,1,1,5,214,6,14,1,1,27:44,19,25,283,2,0,0,0,11.3,11.3,76.0,139.2,34,123,3.6,3,1,1,6,6,3,127,7,13,2,2,32:16
12346,2023,WAS,15,Sun,December 17,L,no,away,Los Angeles Rams,20,28,19,36,218,3,1,1,8,6.3,5.9,52.8,87.5,21,79,3.8,0,0,0,2,3,5,203,3,14,3,5,24:07,25,33,249,2,0,3,9,7.8,6.9,75.8,116.9,37,196,5.3,1,2,3,2,2,3,134,9,16,0,0,35:53
12347,2023,WAS,16,Sun,December 24,L,no,away,New York Jets,28,30,16,35,143,1,2,1,13,4.5,4.0,45.7,42.9,22,102,4.6,3,0,0,4,4,8,294,3,12,1,1,23:44,27,49,217,1,1,0,0,4.4,4.4,55.1,64.8,36,164,4.6,2,3,3,3,3,6,292,4,16,1,2,36:16
12348,2023,WAS,17,Sun,December 31,L,no,home,San Francisco 49ers,10,27,17,28,163,1,2,1,6,6.0,5.6,60.7,59.1,14,62,4.4,0,1,1,1,1,3,156,3,9,0,1,21:47,22,28,224,2,0,1,6,8.2,7.7,78.6,123.8,39,184,4.7,1,2,2,3,3,2,76,4,9,0,1,38:13


In [19]:
pd.read_csv('nfl_data/airports.csv')

,team,airport,latitude,longitude,time_zone
0,ARI,PHX,33.434,-112.008,-2
1,ATL,ATL,33.640,-84.427,0
2,BAL,BWI,39.175,-76.668,0
3,BUF,BUF,42.940,-78.732,0
4,CAR,CLT,35.214,-80.943,0
5,CHI,ORD,41.979,-87.904,-1
6,CIN,CVG,39.046,-84.662,0
7,CLE,CLE,41.412,-81.850,0
8,DAL,DFW,32.896,-97.037,-1
9,DEN,DEN,39.858,-104.667,-2
